### Clasificación de flores

El código mostrado en este ejercicio utiliza las fotos de flores de la carpeta `datos/flower_photos` para entrenar un modelo de clasificación. Las fotos están divididas en subcarpetas con el nombre del tipo de flor (roses, tulips, daisy,...).

Las fotos se cargan en dos conjuntos de datos (datasets) de TensorFlow, uno para el entrenamiento y otro para la validación.

**Objetivos:**

- Conocer algunas de las funciones de TensorFlow para la carga de datos y el preprocesamiento.
- Seguir trabajando con la API de Keras para crear modelos de redes neuronales.

### Tareas del estudiante
1. Activar el entorno **curso-python-ml-rn** y ejecutar el código de todas las celdas.
2. Revisar cómo se cargan las fotos en los datasets.
   1. ¿Qué función se utiliza?
   2. ¿En qué propiedad del dataset se guardan los nombres de las carpetas (clases)?
   3. ¿Qué porcentaje de las fotos se usa para el entrenamiento?
3. Escalado
   1. ¿Cómo se llama la función que se utiliza para hacer el escalado?
   2. ¿En qué otra parte del código está esta función? Aunque está comentada.
4. Creación del modelo
   1. ¿Cuántas capas ocultas tiene la red neuronal?
   2. ¿Cuántas salidas tiene la capa de salida?
   3. Busque en la documentación de TensorFlow qué características tiene cada capa.
5. Compilación
   1. Busque en la documentación de TensorFlow qué significan cada uno de los argumentos que se han pasado a la función `compile`.
6. Entrenamiento
   1. ¿Qué significa el epoch?
   2. Prueba a cambiar el epoch y comprueba si se obtienen mejores resultados en la evaluación.
   3. Hay una función que muestra un resumen del modelo en formato texto. Busque en la documentación de TensorFlow cuál es la función y utilícela en la **celda vacía que está después de entrenar el modelo**.
7. Evaluación del modelo
   1. ¿Crees que son buenos los resultados de la evaluación?
   2. ¿Cómo se llama lo que está sucediendo?
8. Mejora del modelo
   1. **Aumento de datos**: Esta es una técnica para generar más datos de entrenamiento a partir de modificar los que ya existen.
      1. Revise el código que está en las últimas celdas, donde se crea una secuencia con el nombre `data_augmentation`.
      2. Utilice dicha secuencia como el primer paso al crear el modelo de clasificación.
   2. **Dropout**: Es una técnica de regularización de la red neuronal que consiste en que durante el entrenamiento se descarta de manera aleatoria un porcentaje de los datos de salida de una capa.
      1. Al crear el modelo, introduzca una nueva capa `Dropout` justo antes de la capa Flatten y que descarte el 20% de los datos.
9. Vuelve a evaluar el modelo luego de hacer los dos cambios anteriores.
10. Predicción con una nueva foto
    1. Busca en internet una foto de una flor y vaya a la última celda de este notebook y copie en URL y ejecute la celda, para predecir la clase de la foto.

---

In [ ]:
import pathlib
import os
import numpy as np
import PIL
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
# Esta variable de entorno es necesaria para evitar que un error fatal de TensorFlow 
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# Ignorando los warnings de TensorFlow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
# Ruta a la carpeta con las imágenes
IMAGES_DIR_PATH =  'datos/flower_photos'

In [ ]:
# Contando cuantas imágenes hay 
data_dir = pathlib.Path(IMAGES_DIR_PATH)
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

In [ ]:
# Mostrando una rosa
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 180
IMG_WIDTH = 180

In [ ]:
# Creando el dataset de entrenamiento

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size = BATCH_SIZE)

In [ ]:
# Creando el dataset de validación

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size = BATCH_SIZE)

In [ ]:
# Guardando los nombres de las clases en una variable
class_names = train_ds.class_names
class_names

In [ ]:
# Visualizando algunas imágenes
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# Configurando el dataset para un mejor rendimiento
# - Cache de las imágenes
# - Solapando el preprocesamiento y la ejcución al entrenar el modelo

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Escalando los datos para que los valores de cada pixel estén entre 0 y 1. Ahora están entre 0 y 255.

normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
# Creando el modelo con la API secuencial de Keras

num_classes = len(class_names)

model = Sequential([
  # layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
# Compilando el modelo

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Entrenando el modelo

EPOCHS = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

In [ ]:
# Utilice esta celda para llamar a una función que muestre 
# un resumen de la estructura del modelo. Busque el ombre de la función en la documentatción.

In [ ]:
# Visualizando los resultados del entrenamiento

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

**Aumento de datos (Data augmentation)**

In [ ]:
# Definiendo una secuencia para generar nuevas fotos a patrtir de transformar las que existen

data_augmentation = Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(IMG_HEIGHT,
                                  IMG_WIDTH,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
# Haciendo una prueba con la secuencia creada arriba

plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

**Predicción con una foto nueva**

Busque alguna foto de una flor internet, que sea de alguna de las clases de flores que entrenamos, y copie el URL en la celda de abajo y ejecútela.

In [ ]:
FLOWER_URL = ""  #Copie aquí el URL de una foto 
if FLOWER_URL != "":

    flower_path = tf.keras.utils.get_file('Red_sunflower', origin=FLOWER_URL)

    img = tf.keras.utils.load_img(
        flower_path, target_size=(IMG_HEIGHT, IMG_WIDTH)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "Esta imagen parece ser de una {} con una seguridad del {:.2f} %."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )